# Simulation: Passing Percentage

In [1]:
import os, sys, random, statistics
import pandas as PD

In [2]:
BACKEND_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
APPPROJ_PATH = os.path.join(BACKEND_PATH, 'app_proj')
sys.path.append(APPPROJ_PATH)
import notebooks as NT
print([x for x in dir(NT) if x[0]!='_' and len(x)>8])

['DataframeToDicts', 'GetRandom', 'NOTEBOOK_ENV']


In [3]:
MODULE_PATH = os.path.dirname(os.getcwd())
DATA_PATH = os.path.join(MODULE_PATH, 'data')
os.path.exists(DATA_PATH)

True

In [4]:
import django
os.environ['DEPLOYMENT_ENV'] = NT.NOTEBOOK_ENV
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
os.chdir(os.path.dirname(MODULE_PATH))
django.setup()

env: DEV


In [5]:
import emporium.logic.stage as SG
print([x for x in dir(SG) if x[0]!='_' and len(x)>8])

['DIFFICULTY', 'GetDevAgility', 'GetDevCombat', 'GetDevCunning', 'GetDevMight', 'GetDevelopmentTable', 'GetProductionTable', 'ObstacleSequence', 'TowerStageType']


In [6]:
import emporium.logic.simulation as SM
print([x for x in dir(SM) if x[0]!='_' and len(x)>8])

['ApplyRandomLevels', 'AttachWargear', 'RollDamage', 'RunBeatCount', 'RunPassTest']


In [7]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import IPython.display as DS 
from ipywidgets import IntProgress

### Simulation

In [8]:
baseThief = {
    'Agi': 0, 'Cun': 0, 'Mig': 3, 'End': 0,
    'Hlt': 58,
    'Sab': 0, 'Per': 0, 'Tra': 0, 
}
# baseThief['Mig'] += 1
baseThief = SM.AttachWargear(baseThief)
# baseThief['Tra'] += 5
PD.DataFrame([baseThief])

,Agi,Cun,Mig,End,Hlt,Sab,Per,Tra,Att,Dmg,Def
0,0,0,5,0,58,0,0,0,3,6,17


In [9]:
THIEF_LEVEL = 2
thiefConfig = SM.ApplyRandomLevels(baseThief, THIEF_LEVEL)
PD.DataFrame([thiefConfig])

,Agi,Cun,Mig,End,Hlt,Sab,Per,Tra,Att,Dmg,Def
0,1,0,5,0,58,5,0,0,4,6,17


In [10]:
potentialLs = SG.GetDevelopmentTable(1,2,1,1,1)
# potentialLs = SG.GetProductionTable(1,2,1,1,1)
OBS_TEST = 8
obstacleLs = SG.ObstacleSequence(potentialLs, OBS_TEST, 'balanced')
PD.DataFrame(obstacleLs)

,Name,Trait,Skill,Success,Failure,Experience,Attack,Damage,Defense,Health,Difficulty
0,class2,all2,Fight,"experience, pass",knockout,13,3.0,10,13.0,9.0,NaN
1,class3,all3,Fight,"experience, pass",knockout,13,2.0,6,16.0,12.0,NaN
2,Chandelier,Agi,Per,"experience, pass next","wound, pass",8,NaN,11,NaN,NaN,15.0
3,class1,all1,Fight,"experience, pass",knockout,13,2.0,8,14.0,11.0,NaN
4,Spike Trap,Agi,Tra,"experience, pass","wound, pass",8,NaN,11,NaN,NaN,13.0
5,class1,all1,Fight,"experience, pass",knockout,13,2.0,8,14.0,11.0,NaN
6,Crossbow Trap,Cun,Per,"experience, pass","wound, pass",8,NaN,11,NaN,NaN,13.0
7,class1,all1,Fight,"experience, pass",knockout,13,2.0,8,14.0,11.0,NaN


In [11]:
runLs = SM.RunPassTest(thiefConfig, obstacleLs)
PD.DataFrame(runLs)

,obstacle,passed,curr pos,next pos,wounds,xp,gold
0,class2,True,0,1,0,2,0
1,class3,True,1,2,0,4,0
2,Chandelier,False,2,3,6,4,0
3,class1,True,3,4,6,6,0
4,Spike Trap,False,4,5,20,6,0
5,class1,True,5,6,20,8,0
6,Crossbow Trap,False,6,7,34,8,0
7,class1,True,7,8,34,10,0


In [12]:
TRIALS = 800000
clearedLs = []
progressBar = IntProgress(min=0, max=TRIALS/10000) 
DS.display(progressBar) 

for tr in range(0, TRIALS):
    if tr % 10000 == 0: progressBar.value += 1

    thiefConfig = SM.ApplyRandomLevels(baseThief, THIEF_LEVEL)
    obstacleLs = SG.ObstacleSequence(potentialLs, OBS_TEST, 'balanced')

    runLs = SM.RunPassTest(thiefConfig, obstacleLs)
    clearedLs.append( runLs[-1]['next pos'] == OBS_TEST )

round(statistics.mean(clearedLs)*100, 1)

IntProgress(value=0, max=80)

80.9